In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances

In [2]:
# read in embedding_768_TCGA_COAD.csv
data = pd.read_csv('/home/qiuaodon/Desktop/project_data_new/embedding_768_TCGA_COAD.csv')

In [3]:
print(f"Data shape: {data.shape}")

Data shape: (460, 770)


In [4]:
# remove the last two columns
# set the column patient_id as index
data.index = data.iloc[:, -1]
data = data.iloc[:, :-2]

In [ ]:
#data = data.drop(index=0).reset_index(drop=True)


In [ ]:
data

In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import squareform
import random

# Set a seed for reproducibility
random.seed(42)
np.random.seed(42)

# Transpose the data so that features are now samples
data_T = data.T  # Assuming 'data' is a pandas DataFrame

# Parameters
num_clusters_range = range(2, 14)  # Testing cluster counts from 2 to 10
n_resamples = 100  # Number of resampling iterations

# Initialize variables
num_features = data_T.shape[0]  # Number of features (now samples)
feature_names = data_T.index     # Feature names
consensus_matrices = {}          # To store consensus matrices for each cluster count

# Perform consensus clustering on features
for n_clusters in num_clusters_range:
    print(f"Processing {n_clusters} clusters...")
    temp_matrix = np.zeros((num_features, num_features))
    
    for _ in range(n_resamples):
        # Randomly sample 80% of the patients (now features) without replacement
        sample_indices = np.random.choice(data_T.columns, size=int(data_T.shape[1] * 0.8), replace=False)
        sampled_data = data_T.loc[:, sample_indices]
        
        # Apply KMeans clustering to the features
        kmeans = KMeans(n_clusters=n_clusters, n_init=10, random_state=None)
        labels = kmeans.fit_predict(sampled_data)
        
        # Update the co-occurrence matrix
        for i in range(num_features):
            for j in range(num_features):
                if labels[i] == labels[j]:
                    temp_matrix[i, j] += 1
    
    # Normalize the temporary matrix
    temp_matrix /= n_resamples
    # Store the consensus matrix for this cluster count
    consensus_matrices[n_clusters] = temp_matrix

    # Convert the consensus matrix to a distance matrix
    distance_matrix = 1 - temp_matrix

    # Perform hierarchical clustering
    # Since distance_matrix is a square matrix, we need to convert it to a condensed distance matrix
    condensed_distance = squareform(distance_matrix, checks=False)
    linkage_matrix = linkage(condensed_distance, method='average')

    # Assign clusters based on the linkage matrix
    cluster_assignments = fcluster(linkage_matrix, n_clusters, criterion='maxclust')

    # Create a DataFrame to save the assignments
    assignments_df = pd.DataFrame({
        'Feature': feature_names,
        'Cluster': cluster_assignments
    })

    # Save to CSV
    #assignments_df.to_csv(f'cluster_assignments_{n_clusters}_clusters.csv', index=False)


In [ ]:
def calculate_pac(consensus_matrix, lower_bound=0.1, upper_bound=0.9):
    import numpy as np
    consensus_values = consensus_matrix.flatten()
    pac = np.mean((consensus_values > lower_bound) & (consensus_values < upper_bound))
    return pac

pac_scores = {}
for n_clusters, consensus_matrix in consensus_matrices.items():
    pac_score = calculate_pac(consensus_matrix)
    pac_scores[n_clusters] = pac_score
    print(f'PAC score for {n_clusters} clusters: {pac_score}')


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# Function to plot consensus matrices for multiple cluster numbers
def plot_consensus_matrices(consensus_matrices, cluster_range):
    for n_clusters in cluster_range:
        # Get the consensus matrix for the given number of clusters
        consensus_matrix = consensus_matrices[n_clusters]

        # Perform KMeans clustering on the consensus matrix
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        cluster_labels = kmeans.fit_predict(consensus_matrix)

        # Reorder the consensus matrix and feature indices by the cluster labels
        sorted_indices = np.argsort(cluster_labels)
        sorted_matrix = consensus_matrix[sorted_indices, :][:, sorted_indices]
        sorted_feature_indices = np.array(range(len(consensus_matrix)))[sorted_indices]  # Reorder feature indices

        # Plot the reordered consensus matrix
        plt.figure(figsize=(10, 8))
        sns.heatmap(sorted_matrix, cmap='viridis', xticklabels=sorted_feature_indices, yticklabels=sorted_feature_indices)
        plt.title(f'Consensus Matrix for {n_clusters} Clusters')
        plt.show()

# Define the range of cluster numbers you want to plot
cluster_range = range(2, 14)

# Plot consensus matrices for clusters 2 to 10
plot_consensus_matrices(consensus_matrices, cluster_range)


In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

def plot_elbow_method(data, cluster_range):
    inertia = []
    
    # Compute inertia for different numbers of clusters
    for n_clusters in cluster_range:
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        kmeans.fit(data)
        inertia.append(kmeans.inertia_)
    
    # Plot inertia vs. number of clusters
    plt.figure(figsize=(8, 6))
    plt.plot(cluster_range, inertia, marker='o')
    plt.title('Elbow Method for Optimal Clusters')
    plt.xlabel('Number of Clusters')
    plt.ylabel('Inertia (Sum of Squared Distances)')
    plt.xticks(cluster_range)
    plt.show()

# Define the range of cluster numbers to test
cluster_range = range(2, 14)

# Call the function using your data (assuming `data` is your dataset)
plot_elbow_method(data, cluster_range)


In [ ]:
from sklearn.metrics import silhouette_score

def plot_silhouette_score(data, cluster_range):
    silhouette_avg = []
    
    # Compute the silhouette score for different numbers of clusters
    for n_clusters in cluster_range:
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        cluster_labels = kmeans.fit_predict(data)
        silhouette_avg.append(silhouette_score(data, cluster_labels))
    
    # Plot silhouette score vs. number of clusters
    plt.figure(figsize=(8, 6))
    plt.plot(cluster_range, silhouette_avg, marker='o')
    plt.title('Silhouette Score for Optimal Clusters')
    plt.xlabel('Number of Clusters')
    plt.ylabel('Silhouette Score')
    plt.xticks(cluster_range)
    plt.show()

# Call the function using your data (assuming `data` is your dataset)
plot_silhouette_score(data, cluster_range)


In [ ]:
from sklearn.metrics import calinski_harabasz_score

def plot_calinski_harabasz(data, cluster_range):
    ch_scores = []
    
    # Compute the Calinski-Harabasz index for different numbers of clusters
    for n_clusters in cluster_range:
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        cluster_labels = kmeans.fit_predict(data)
        ch_scores.append(calinski_harabasz_score(data, cluster_labels))
    
    # Plot the Calinski-Harabasz index vs. number of clusters
    plt.figure(figsize=(8, 6))
    plt.plot(cluster_range, ch_scores, marker='o')
    plt.title('Calinski-Harabasz Index for Optimal Clusters')
    plt.xlabel('Number of Clusters')
    plt.ylabel('Calinski-Harabasz Index')
    plt.xticks(cluster_range)
    plt.show()

# Call the function using your data
plot_calinski_harabasz(data, cluster_range)


In [ ]:
from sklearn.mixture import GaussianMixture

def plot_bic_aic(data, cluster_range):
    bic_scores = []
    aic_scores = []
    
    # Compute BIC and AIC for different numbers of clusters using GMM
    for n_clusters in cluster_range:
        gmm = GaussianMixture(n_components=n_clusters, random_state=42)
        gmm.fit(data)
        bic_scores.append(gmm.bic(data))
        aic_scores.append(gmm.aic(data))
    
    # Plot BIC and AIC vs. number of clusters
    plt.figure(figsize=(8, 6))
    plt.plot(cluster_range, bic_scores, marker='o', label='BIC')
    plt.plot(cluster_range, aic_scores, marker='o', label='AIC')
    plt.title('BIC/AIC for Optimal Clusters')
    plt.xlabel('Number of Clusters')
    plt.ylabel('Score')
    plt.legend()
    plt.xticks(cluster_range)
    plt.show()

# Call the function using your data
plot_bic_aic(data, cluster_range)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def calculate_pac(consensus_matrix, lower_bound=0.1, upper_bound=0.9):
    """
    Calculate the Proportion of Ambiguous Clustering (PAC) score.

    Parameters:
    - consensus_matrix: The consensus matrix from consensus clustering.
    - lower_bound: The lower threshold for ambiguity (default is 0.1).
    - upper_bound: The upper threshold for ambiguity (default is 0.9).

    Returns:
    - pac_score: The PAC score, where a lower value indicates more stable clustering.
    """
    # Flatten the consensus matrix into a 1D array of values
    consensus_values = consensus_matrix.flatten()

    # Count the proportion of values between lower_bound and upper_bound
    ambiguous_proportion = np.mean((consensus_values > lower_bound) & (consensus_values < upper_bound))

    return ambiguous_proportion

def plot_pac_scores(consensus_matrices, cluster_range):
    """
    Plot the PAC scores for different numbers of clusters.

    Parameters:
    - consensus_matrices: A dictionary of consensus matrices, where the key is the number of clusters.
    - cluster_range: The range of cluster numbers to compute PAC scores for.
    """
    pac_scores = []

    # Calculate PAC scores for each cluster number
    for n_clusters in cluster_range:
        consensus_matrix = consensus_matrices[n_clusters]
        pac_score = calculate_pac(consensus_matrix)
        pac_scores.append(pac_score)
        print(f'PAC score for {n_clusters} clusters: {pac_score}')

    # Plot PAC scores
    plt.figure(figsize=(8, 6))
    plt.plot(cluster_range, pac_scores, marker='o', linestyle='-', color='b')
    plt.title('PAC Score for Different Number of Clusters')
    plt.xlabel('Number of Clusters')
    plt.ylabel('PAC Score')
    plt.xticks(cluster_range)
    plt.grid(True)
    plt.show()

# Example usage:
# Assuming `consensus_matrices` is a dictionary with cluster numbers as keys and consensus matrices as values
cluster_range = range(2, 14)  # For clusters 2 through 10
plot_pac_scores(consensus_matrices, cluster_range)
